In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
train_dataset=pd.read_csv('dataset/train_dataset.csv', index_col='date')
validation_dataset=pd.read_csv('dataset/validation_dataset.csv',index_col='date')

In [3]:
X=train_dataset.iloc[:,1:-2]
y=train_dataset.target

In [4]:
train=0.9
test=0.1
era=len(train_dataset.index.unique())
train_era=int(era*train)
test_era=era-train_era
X_train=X[X.index<X.index.unique()[train_era]]
y_train=y[y.index<y.index.unique()[train_era]]
X_test=X[X.index>=X.index.unique()[train_era]]
y_test=y[y.index>=y.index.unique()[train_era]]

In [5]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
# Define error metrics
from sklearn.metrics import mean_squared_error

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


In [9]:
xgreg1= XGBRegressor(n_estimators=1000, max_depth=4, eta=0.1,
                     subsample=0.9, colsample_bytree=0.2,learning_rate=0.1,gamma=0.25)

In [10]:
xgreg1=xgreg1.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric='rmse',eval_set=[(X_test,y_test)])

[0]	validation_0-rmse:0.28883
[1]	validation_0-rmse:0.28834
[2]	validation_0-rmse:0.28808
[3]	validation_0-rmse:0.28778
[4]	validation_0-rmse:0.28757
[5]	validation_0-rmse:0.28738
[6]	validation_0-rmse:0.28720
[7]	validation_0-rmse:0.28700
[8]	validation_0-rmse:0.28693
[9]	validation_0-rmse:0.28680
[10]	validation_0-rmse:0.28677
[11]	validation_0-rmse:0.28669
[12]	validation_0-rmse:0.28661
[13]	validation_0-rmse:0.28654
[14]	validation_0-rmse:0.28650
[15]	validation_0-rmse:0.28652
[16]	validation_0-rmse:0.28649
[17]	validation_0-rmse:0.28643
[18]	validation_0-rmse:0.28638
[19]	validation_0-rmse:0.28641
[20]	validation_0-rmse:0.28638
[21]	validation_0-rmse:0.28638
[22]	validation_0-rmse:0.28634
[23]	validation_0-rmse:0.28633
[24]	validation_0-rmse:0.28629
[25]	validation_0-rmse:0.28626
[26]	validation_0-rmse:0.28627
[27]	validation_0-rmse:0.28624
[28]	validation_0-rmse:0.28617
[29]	validation_0-rmse:0.28619
[30]	validation_0-rmse:0.28618
[31]	validation_0-rmse:0.28618
[32]	validation_0-

In [11]:
y_pred_train = xgreg1.predict(X_train)
print('rmse for train dataset: {}'.format(rmse(y_train,y_pred_train)))

y_pred_test = xgreg1.predict(X_test)
print('rmse for test dataset: {}'.format(rmse(y_test,y_pred_test)))

rmse for train dataset: 0.27646204689870624
rmse for test dataset: 0.2859118224537913


In [12]:
predictionXG1 = pd.DataFrame()
predictionXG1['date'] = list(X_test.index)
predictionXG1.set_index(['date'],inplace=True)
predictionXG1['prediction'] = y_pred_test
predictionXG1['label']=y_test
predictionXG1['delta']=predictionXG1.prediction-predictionXG1.label
predictionXG1

,prediction,label,delta
date,,,
2021-05-30,0.555663,0.297571,0.258092
2021-05-30,0.487396,0.022267,0.465128
2021-05-30,0.559689,0.265182,0.294507
2021-05-30,0.533107,0.500000,0.033107
2021-05-30,0.524820,0.305668,0.219152
...,...,...,...
2021-09-26,0.336473,0.602972,-0.266500
2021-09-26,0.499980,0.477707,0.022273
2021-09-26,0.418128,0.259023,0.159104


In [13]:
# Use the trained Logistic Regression model to make prediction on latest data

X_validation = validation_dataset.iloc[:, 1:-2]
y_validation = xgreg1.predict(X_validation)

#y_validation

In [14]:
prediction = pd.DataFrame()
prediction['symbol'] = list(validation_dataset.symbol)
prediction['prediction'] = y_validation
prediction.head()

,symbol,prediction
0,SALT,0.526240
1,XRP,0.525340
2,PRE,0.473191
3,QSP,0.474419
4,PAY,0.533491


In [15]:
#check if y_latest is in accordance to shape for submission
if set(prediction.symbol) == set(validation_dataset.symbol):
    print('symbol matched!')
else:
    print('symbol unmatched, the symbol in prediction df must match the symbol in validation_dataset...')
    
if prediction.shape[1] == 2:
    print('column counts ok!')
else:
    print('It should have 2 columns, first column with symbol, second with prediction...')


symbol matched!
column counts ok!


In [16]:
#Output submission as .csv file
prediction.to_csv('prediction/prediction.csv', index = False, header = None)

In [17]:
prediction.prediction.min()

0.23198336362838745

In [18]:
prediction.prediction.max()

0.6875046491622925